#### 개정 일자: 2020.11.02


#### 2020.11.02

1) 뉴스검색결과 전체 건수 항목 사라져서 삭제

#### 2020.05.02
1) 시작날짜, 종료날짜 입력하여 실행하도록 설정

2) column단위 병합에서 row 단위 개별 추가로 변경

In [1]:
import re
import time
import requests
import pandas as pd
from pandas import DataFrame
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm_notebook

In [2]:
#수집 : keyword/ 정렬순서(최신/과거)/ 날짜 지정 

def searchNewsExtractor(keyword,sdate,edate): #자동
    global df
    df = pd.DataFrame(columns = ["날짜", "언론사", "제목", "본문", "url"])

    error_cnt= 0
    dot_sdate = sdate[0:4]+"."+sdate[4:6]+"."+sdate[6:8]
    dot_edate = edate[0:4]+"."+edate[4:6]+"."+edate[6:8]
    
    for i in tqdm_notebook(range(50)): #4,000건이 최대
        pageNo= 10*i+1
        url = "https://search.naver.com/search.naver?&where=news&query="+keyword+"&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=3&ds="+dot_sdate+"&de="+dot_edate+"&docid=&nso=so:dd,p:from"+sdate+"to"+edate+",a:all&mynews=0&start="+str(pageNo)+"&refresh_start=0"
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'lxml')

        if soup.find("div",{"class":"not_found02"}):
            break 
        else:
            df = pd.concat([df,newsParser(soup, df)], ignore_index=True)
            
    return df


def newsParser(soup,df):
    news_boxs = soup.find_all("div",{"class":"news_area"})
    df_box = pd.DataFrame(columns = ["날짜", "언론사", "제목", "본문", "url"])
    for box in news_boxs:
        press = box.find("a", {"class": "info press"}).text.replace("언론사 선정","").strip() #언론사
        url = box.find("a", {"class":"news_tit"})['href'] #url
        title = box.find("a", {"class":"news_tit"}).text.strip() #제목
        date = box.find_all("span", {"class": "info"}) #날짜
        content = box.find("div", {"class":"news_dsc"}).text.strip() #내용
    
        #-시간/-일 전 정제하기
        if len(date) ==1: 
            date = date[0].text
        else: 
            date = date[1].text
            
        try:
            match = re.search(r'\d{4}.\d{2}.\d{2}', date)
            date = str(match.group())
        except AttributeError:
            try: 
                if "일" in date:
                    match = re.search('\d+일 전', date)
                    date = str(match.group())
                elif "시간" in date:
                    match = re.search('\d+시간 전', date)
                    date = str(match.group())
                else: pass
            except AttributeError: 
                pass
            
        #print("press:{} \n \n href:{} \n \n title:{} \n \n date:{} \n \n content:{}".format(press,url, title, date, content))
        row = pd.Series([date, press, title, content, url], index = ["날짜", "언론사", "제목", "본문", "url"])
        df_box = df_box.append(row, ignore_index=True)
        
    return df_box
    
def to_excel(filenm):

    for i in range(len(df)):
        #최신날짜 "-일전/-시간 전" 정제하기 
        if "일" in df['날짜'][i]:
            match = re.search('\d+', df['날짜'][i])
            now = datetime.now() #현재 시간(수집 시간)
            diff = int(match.group()) #몇일 전/몇시간 전인지
            date = now - timedelta(days=diff)
            df['날짜'][i] = date.strftime("%Y.%m.%d")

        elif "시간" in df['날짜'][i]:
            match = re.search('\d+', df['날짜'][i])
            now = datetime.now() #현재 시간(수집 시간)
            diff = int(match.group()) #몇일 전/몇시간 전인지
            date = now - timedelta(hours=diff)
            df['날짜'][i] = date.strftime("%Y.%m.%d")
            #print("몇시간전? :",diff,"\n현재시각: ",now, "\n바뀐 시간: ", date)
            #print("입력시간: ",date.strftime("%Y.%m.%d"))
        else: pass
    
    df.to_excel(filenm, index=True)
    return df

In [4]:
#####-------------main-----------------#####
keyword = "스트래티지애널리틱스" #카운터포인트리서치/스트래티지애널리틱스
sdate = "20210201" #시작 날짜
edate = "20210228" #종료 날짜 
filenm = keyword+"_"+sdate+"_"+edate+".xlsx" #저장할 파일명

#1. 최신순/과거순
#searchNewsExtractor1(keyword,sortNo)

#2. 날짜지정
df = searchNewsExtractor(keyword,sdate,edate)
to_excel(filenm)

,날짜,언론사,제목,본문,url
0,2021.02.28,경향신문,“삼성폰 세계 1위를 지켜라” 국가별 맞춤전략 골몰,미국 시장조사업체 스트래티지 애널리틱스(SA)에 따르면 삼성전자의 일본 스마트폰 시...,http://news.khan.co.kr/kh_news/khan_art_view.h...
1,2021.02.28,머니S,"우리가 꿈꾸던 가상·증강현실, 언제쯤 현실로 다가올까",스트래티지애널리틱스(SA)가 미국 소비자를 대상으로 진행한 설문조사에서도 VR·AR...,http://moneys.mt.co.kr/news/mwView.php?no=2021...
2,2021.02.27,한국경제,'원조 노캔' 보스 무선이어폰 'QC 이어버드' 써보니 [배성수의 다다IT선],시장조사업체 스트래티지 애널리틱스(SA)에 따르면 내년 무선이어폰(TWS) 시장 규...,https://www.hankyung.com/it/article/202102266638g
3,2021.02.26,데일리한국,"삼성전자, 올해 스마트폰 ODM 물량 얼마나 늘리나",시장조사업체 스트래티지애널리틱스(SA)에 따르면 지난해 삼성전자 스마트폰 총 출하량...,https://daily.hankooki.com/lpage/ittech/202102...
4,2021.02.26,EBN,"""접는 스마트폰 시대""…삼성 폴더블폰 대중화 주도",지난해 스마트폰 출하량(스트래티지 애널리틱스 조사)인 12억9990만대와 비교하면 ...,https://www.ebn.co.kr/news/view/1473656/?sc=Naver
...,...,...,...,...,...
84,2021.02.01,블로터,"애플, 日 무선 이어폰 시장 ‘독주’…소니는 안방서 3위 이하로",27일(현지시각) 시장조사기관 스트래티지애널리틱스(SA)에 따르면 지난해 전 세계 ...,http://www.bloter.net/archives/519041
85,2021.02.01,시사위크,코로나 '집콕'에 태블릿 '싱글벙글'… 7년 만에 최대 성장,글로벌 시장조사업체 스트래티지 애널리틱스(SA)는 1월 30일 지난해 전 세계 태블...,http://www.sisaweek.com/news/articleView.html?...
86,2021.02.01,아시아투데이,전장시장 불붙었는데…이재용이 9조원에 인수한 하만 경쟁력 언제 찾나,시장조사업체 스트래티지 애널리틱스에 따르면 전장 시장 규모는 2015년 2390억달...,https://www.asiatoday.co.kr/view.php?key=20210...
87,2021.02.01,조선비즈,"삼성전자 ‘만년 2인자’로 남나…세계 1위 애플에 밀리고, 인텔에 치이고",시장조사업체 스트래티지 애널리틱스(SA)에 따르면 올해 5세대 이동통신(5G) 스마...,https://biz.chosun.com/site/data/html_dir/2021...
